<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/1_G%C3%9CN_15_DAK_MUM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from rich.console import Console
from rich.table import Table
from datetime import datetime, timedelta

# 1. VERİ ALMA FONKSİYONU
def get_data(symbols, period='1d', interval='15m'):
    data = {}
    for symbol in symbols:
        ticker = yf.Ticker(symbol)
        df = ticker.history(period=period, interval=interval)
        data[symbol] = df
    return data

# 2. TEKNİK GÖSTERGE HESAPLAMALARI
def calculate_indicators(df, params):
    # RSI Hesapla
    def rsi(series, period):
        delta = series.diff()
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)

        avg_gain = gain.rolling(period).mean()
        avg_loss = loss.rolling(period).mean()

        rs = avg_gain / avg_loss
        return 100 - (100 / (1 + rs))

    # MFI Hesapla
    def mfi(high, low, close, volume, period):
        typical_price = (high + low + close) / 3
        raw_money_flow = typical_price * volume

        positive_flow = raw_money_flow.where(typical_price > typical_price.shift(1), 0)
        negative_flow = raw_money_flow.where(typical_price < typical_price.shift(1), 0)

        ratio = positive_flow.rolling(period).sum() / negative_flow.rolling(period).sum()
        return 100 - (100 / (1 + ratio))

    # EMA Delta Hesapla
    ema_short = df['Close'].ewm(span=params['ema_short']).mean()
    ema_long = df['Close'].ewm(span=params['ema_long']).mean()
    delta_ema = ema_short - ema_long

    # RSI Değişim ve Sinyaller
    log_close = np.log(df['Close'])
    rsi_close = rsi(log_close, 14)
    rsi_change = rsi_close.diff()

    signals = {
        'C20L': (rsi_change > 20).astype(int),
        'C10L': (rsi_change > 10).astype(int),
        'C20S': (rsi_change < -20).astype(int),
        'C10S': (rsi_change < -10).astype(int)
    }

    return {
        'RSI': rsi(df['Close'], params['rsi_length']).iloc[-1],
        'MFI': mfi(df['High'], df['Low'], df['Close'], df['Volume'], params['mfi_length']).iloc[-1],
        'Delta EMA': delta_ema.iloc[-1],
        'RSI Change': rsi_change.iloc[-1],
        'Price': df['Close'].iloc[-1],
        **{k:v.iloc[-1] for k,v in signals.items()}
    }

# 3. TABLO OLUŞTURMA
def create_table(data, params):
    console = Console()
    table = Table(title="C20MultiMonitoringIndTable", show_header=True, header_style="bold magenta")

    # Sütunları ekle
    columns = ['Symbol', 'Price', 'RSI', 'MFI', 'Delta EMA', 'RSI Change',
               'C20L', 'C10L', 'C20S', 'C10S']
    for col in columns:
        table.add_column(col)

    # Satırları ekle
    for symbol, values in data.items():
        row = [
            symbol,
            f"{values['Price']:.2f}",
            f"{values['RSI']:.2f}",
            f"{values['MFI']:.2f}",
            f"{values['Delta EMA']:.4f}",
            f"{values['RSI Change']:.2f}",
            "✓" if values['C20L'] else "",
            "✓" if values['C10L'] else "",
            "✓" if values['C20S'] else "",
            "✓" if values['C10S'] else ""
        ]
        table.add_row(*row)

    console.print(table)

# 4. ANA İŞLEM
def main():
    # Parametreler
    params = {
        'rsi_length': 14,
        'mfi_length': 14,
        'ema_short': 10,
        'ema_long': 34
    }

    symbols = [
        'BTC-USD', 'ETH-USD', 'BNB-USD', 'XRP-USD', 'SOL-USD',
        'ADA-USD', 'DOGE-USD', 'MATIC-USD', 'DOT-USD', 'LTC-USD'
    ]

    # Verileri al
    data = get_data(symbols, period='1d', interval='15m')

    # Göstergeleri hesapla
    results = {}
    for symbol, df in data.items():
        results[symbol] = calculate_indicators(df, params)

    # Tabloyu oluştur
    create_table(results, params)

if __name__ == "__main__":
    main()

                                  C20MultiMonitoringIndTable                                  
┏━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┓
┃ Symbol    ┃ Price     ┃ RSI   ┃ MFI   ┃ Delta EMA ┃ RSI Change ┃ C20L ┃ C10L ┃ C20S ┃ C10S ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━┩
│ BTC-USD   │ 101880.59 │ 19.40 │ 2.28  │ -163.3971 │ -7.61      │      │      │      │      │
│ ETH-USD   │ 3261.89   │ 28.72 │ 47.38 │ -13.1157  │ -1.38      │      │      │      │      │
│ BNB-USD   │ 672.64    │ 15.19 │ 13.69 │ -1.9578   │ -0.74      │      │      │      │      │
│ XRP-USD   │ 3.02      │ 29.60 │ 14.03 │ -0.0089   │ -4.41      │      │      │      │      │
│ SOL-USD   │ 229.24    │ 19.34 │ 34.84 │ -0.9332   │ -4.61      │      │      │      │      │
│ ADA-USD   │ 0.93      │ 12.38 │ 26.91 │ -0.0058   │ -12.52     │      │      │      │ ✓    │
│ DOGE-USD  │ 0.32      │ 23.44 │ 44.66 │ -0.0016   │ 1.81       │      │      │      │      │
│ MATIC-USD │ 0.40      │ 3.77  │ 13.80 │ -0.0021   │ -11.28     │      │      │      │ ✓    │
│ DOT-USD   │ 6.39      │ 43.26 │ 79.32 │ -0.0055   │ -5.35      │      │      │      │      │
│ LTC-USD   │ 124.32    │ 21.07 │ 9.45  │ -1.6877   │ -0.36      │      │      │      │      │
└───────────┴───────────┴───────┴───────┴───────────┴────────────┴──────┴──────┴──────┴──────┘